### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [7]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
!pip install langchain_groq

  Using cached langsmith-0.4.4-py3-none-any.whl.metadata (15 kB)
  Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
Using cached langsmith-0.4.4-py3-none-any.whl (367 kB)
Using cached zstandard-0.23.0-cp312-cp312-win_amd64.whl (495 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.129
    Uninstalling langsmith-0.1.129:
      Successfully uninstalled langsmith-0.1.129
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.10
    Uninstalling langchain-core-0.3.10:
      Successfully uninstalled langchain-core-0.3.10


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.1 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.4.4 which is incompatible.
langchain-community 0.3.1 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.4.4 which is incompatible.


In [5]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000011F7F478470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000011F7F4DE720>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [17]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 26, 'total_tokens': 37, 'completion_time': 0.022, 'prompt_time': 0.003249601, 'queue_time': None, 'total_time': 0.025249601}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_e238d2cf52', 'finish_reason': 'stop', 'logprobs': None}, id='run-88f30adf-3746-488d-ad12-13bf16f1d259-0', usage_metadata={'input_tokens': 26, 'output_tokens': 11, 'total_tokens': 37})

In [18]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [19]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [20]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [22]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [23]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:'),
 HumanMessage(content='Hello')]

In [24]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'